<a href="https://colab.research.google.com/github/RedaPengam/Boavizta-Toolkit/blob/main/Boavizta%20data%20repository%20toolkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**📌 Introduction**

---

Ce notebook a pour objectif de simplifier la recherche dans la base de données open source Boavizta qui rassemble une bonne partie des fiches PCF (Product Carbon Footprint) issues des sites constructeurs : https://github.com/Boavizta/environmental-footprint-data/blob/main/boavizta-data-us.csv

L'empreinte carbone du matériel établie sur sa durée correspond à la colonne "gwp_total" ; son empreinte de fabrication uniquement correspond au produit de la colonne précédente avec "gwp_manufacturing_ratio".

#**🔎 Recherche rapide**

---


1. Exécuter la cellule de code ci-dessous via le bouton play en haut à gauche lors du survol de la cellule de code.
2. Cliquer sur 🪄 dans la cellule des résultats après l'exécution.
3. Cliquer sur "filter" dans la cellule des résultats.

In [1]:
from sys import dont_write_bytecode
import pandas as pd
import requests
import io

url_boavizta = "https://raw.githubusercontent.com/Boavizta/environmental-footprint-data/main/boavizta-data-us.csv"
# téléchargement des data raw
download = requests.get(url_boavizta).content
# filtrage des colonnes souhaitées
listCol_boavizta_crop = ["manufacturer", "name", "subcategory", "hard_drive", "lifetime", "gwp_total", "gwp_use_ratio", "gwp_manufacturing_ratio"]
df = pd.read_csv(io.StringIO(download.decode('utf-8')), usecols = listCol_boavizta_crop)
df

,manufacturer,name,subcategory,gwp_total,gwp_use_ratio,lifetime,gwp_manufacturing_ratio,hard_drive
0,Apple,13-inch MacBook Air (M1 CPU) 256GB - 2020,Laptop,161.000,0.1500,3.0,0.76,SSD 256GB
1,Apple,13-inch MacBook Air (M1 CPU) 512GB - 2020,Laptop,181.000,0.1500,3.0,0.76,SSD 512GB
2,Apple,13-inch MacBook Air with Retina display (1.1GH...,Laptop,174.000,0.1500,3.0,0.77,SSD 256GB
3,Apple,13-inch MacBook Air with Retina display (1.1GH...,Laptop,202.000,0.1500,3.0,0.77,SSD 512GB
4,Apple,13-inch MacBook Pro (1.4GHz quad-core CPU) 256...,Laptop,217.000,0.1700,3.0,0.76,SSD 256GB
...,...,...,...,...,...,...,...,...
1221,Seagate,Nytro 1551 XA1920ME10083 1.92 TB,Hard drive,53.760,0.8570,5.0,NaN,1.92TB SSD
1222,Seagate,Nytro 3331 XS7680SE70004 7.68TB,Hard drive,195.072,0.7790,5.0,NaN,NaN
1223,Seagate,OneStor SP2584 Extensible Storage Platform,SAN/NAS,190000.000,0.9652,10.0,NaN,NaN
1224,Seagate,Pulsar 2 800GB SSD,Hard drive,205.000,0.6600,3.0,NaN,NaN


#**⚙️ Opérations rapides**

---

In [3]:
# existing subcategories
df_subcategories = df["subcategory"].unique()
df_subcategories

array(['Laptop', 'Desktop', 'Monitor', 'EntertainmentT', 'IoT', 'Tablet',
       'Smartphone', 'Entertainment', 'Workstation', 'Server',
       'Thin Client', 'Thin client', 'Multimedia', 'Network', 'Gaming',
       'Converged Edge', 'Converged', 'Printer', 'Hard drive', 'SAN/NAS'],
      dtype=object)

In [4]:
# moyenne des gwp_total selon catégorie x
df_avg_gwp_total = df.query('subcategory == "Server"')["gwp_total"].mean()
print(round(df_avg_gwp_total))

8489


In [7]:
# moyenne des gwp_manufacturing selon catégorie x
df['gwp_manuf'] = df['gwp_total'] * df['gwp_manufacturing_ratio']
df_avg_gwp_manuf_x = df.query('subcategory == "Desktop"')['gwp_manuf'].mean()
df_avg_gwp_manuf_x

283.4194012738854

In [17]:
# moyenne des gwp_manufacturing des chromebooks

import math

df1 = df.query('subcategory == "Laptop"')

# get only chromebooks
df1['name'] = df1['name'].map(lambda name: name.lower())
df2 = df1[df1["name"].str.contains("chromebook")]

# fill gwp_manufacturing_ratio NaN values with 0.75
df2['gwp_manufacturing_ratio'].fillna(0.75, inplace=True)

df2['gwp_manuf'] = df2['gwp_total'] * df2['gwp_manufacturing_ratio']
a = df2['gwp_manuf'].mean()
print(a)

204.16080000000002


<ipython-input-17-1d01bc888338>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['name'] = df1['name'].map(lambda name: name.lower())
<ipython-input-17-1d01bc888338>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['gwp_manufacturing_ratio'].fillna(0.75, inplace=True)
<ipython-input-17-1d01bc888338>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm